# Introduction to SQLAlchemy ORM

*Bill Adams @ DerbyPy - March 2017*

### What is an ORM?

* "Object-Relational Mapping"
* An object-oriented wrapper around tabular data
* Commonly use to represent tables in SQL databases

### Why use an ORM?

* Provides a more idiomatic interface to database
* Reduces the need for string queries mixed in with source code
* Hides differences between DB backends
* Allows dynamic query building without string manipulation
* Can help reduce risk of SQL injection

### Some Benefits of SQLAlchemy ORM

* More flexible than other ORMs
* Expressions resemble SQL
* Extremely extensible and customizable
* Integration with many libraries/frameworks

## Connect to DB and Setup Session

In [ ]:
import sqlalchemy as sa
from sqlalchemy.ext.declarative import declarative_base

# Create engine. This defines how you connect to your database
# `echo=True` will cause SQLAlchemy to print all queries sent to
# the DB (useful for debugging when starting out)
engine = sa.create_engine('sqlite:///music.db', echo=True)

# Create a session. The session tracks and manages the ORM objects, handles
# synchronization with the database, and manages the transaction
SessionMaker = sa.orm.sessionmaker(bind=engine)
session = SessionMaker()

# Base class for our ORM entities
Base = declarative_base()

## About Sessions & Transactions

* All queries and statements run in a transaction
* Each session maintains its own open transaction at all times
* Session's transaction lasts until `.commit()` or `.rollback()` call (then a new transaction is started)
* Session maintains caches of new/modified/deleted ORM objects and only updates the DB when necessary

## Define Schema

In [ ]:
class Band(Base):
    # Name of database table
    __tablename__ = 'bands'

    # Primary key column. All ORM classes should have a primary key
    id = sa.Column(sa.Integer, primary_key=True)

    # A VARCHAR(255) column with NOT NULL and UNIQUE constraints
    name = sa.Column(sa.Unicode(255), nullable=False, unique=True)

    def __repr__(self):
        return '<Band id={0.id} name={0.name}>'.format(self)


class Album(Base):
    __tablename__ = 'albums'

    id = sa.Column(sa.Integer, primary_key=True)

    title = sa.Column(sa.Unicode(255), nullable=False)
    release_date = sa.Column(sa.Date)
    
    # A FK to the bands table
    band_id = sa.Column(sa.ForeignKey(Band.id, ondelete='cascade'), nullable=False)

    # A relationship allowing bi-directional linking between band
    # and album python objects
    # In this case the FK relationship between the two tables is 
    # unambiguious so SQLA can detect it automatically
    band = sa.orm.relationship(Band, backref='albums')

    def __repr__(self):
        return '<Album id={0.id} title={0.title} band={0.band}>'.format(self)
        

class Song(Base):
    __tablename__ = 'songs'

    id = sa.Column(sa.Integer, primary_key=True)

    title = sa.Column(sa.Unicode(255), nullable=False)
    track_num = sa.Column(sa.Integer, nullable=False)
    length_seconds = sa.Column(sa.Float, nullable=False)

    album_id = sa.Column(sa.ForeignKey(Album.id, ondelete='cascade'), nullable=False)

    album = sa.orm.relationship(Album, backref=sa.orm.backref('songs', order_by=track_num))

    def __repr__(self):
        return '<Song id={0.id} track_num={0.track_num} album={0.album}>'.format(self)
    
# Create all missing tables for ORM classes inheriting from Base
Base.metadata.create_all(engine)

## Retrieving Records From the Database

In [ ]:
# Retrieve a single record by ID
band = session.query(Band).get(1)
print('Band with ID=1:', band)

In [ ]:
# Accessing column data
# Band variable can be treated like any other python object
print('Band Name: ', band.name)

# Access band's albums
print('Band Albums: ')
for album in band.albums:
    print(album)


In [ ]:
# Querying for all bands
all_bands = session.query(Band).all()
print('All Bands: ', all_bands)

In [ ]:
# Filtering query
the_beatles = session.query(Band).filter(Band.name == 'The Beatles').first()
print('The Beatles: ', the_beatles)

In [ ]:
# More filtering
the_bands = session.query(Band).filter(Band.name.like('%The%')).all()
print('Bands with "The" in their name: ', the_bands)

In [ ]:
# Boolean expression
session.query(Band).filter(
    sa.or_(Band.name == 'The Beatles', Band.name.like('%Simon%'))
).all()

In [ ]:
# Another way of writing the same query
session.query(Band).filter(
    (Band.name == 'The Beatles') | Band.name.like('%Simon%')
).all()

## Joins

In [ ]:
session.query(
    Band, 
    Album
).join(
    Album,
    Album.band_id == Band.id
).all()

In [ ]:
# The same query using the join condition in the relationship
session.query(
    Band, 
    Album
).join(
    Album.band
).all()

## More advanced queries

In [ ]:
# Selecting specific fields

# Looking a lot like SQL at this point
query = session.query(
    # Select clause
    Band.name,
    Album.title,
    Song.track_num,
    Song.title
).select_from(
    Band
).join(
    # Join clause 1
    Album,
    Album.band_id == Band.id
).join(
    # Join clause 2
    Song,
    Song.album_id == Album.id
).filter(
    # Where clause
    Song.length_seconds > 300  # tracks longer than 5 minutes
).order_by(
    Band.name,
    Album.title,
    Song.track_num
)

for result in query:
    print(result)

In [ ]:
# Aggregation
query = session.query(
    Band.name,
    Album.title,
    sa.func.count(Song.id).label('num_tracks'),
    sa.func.sum(Song.length_seconds / 60).label('album_length_mins')
).join(
    Band.albums
).join(
    Album.songs
).group_by(
    Band.name,
    Album.title,
).order_by(
    Band.name,
    Album.title
)

results = query.all()
results

In [ ]:
# Subquery
track_counts_by_band = session.query(
    Band.id.label('band_id'),
    sa.func.count(Song.id).label('song_count')
).join(
    Band.albums
).join(
    Album.songs
).group_by(
    Band.id
).subquery('by_band')

track_count_total = session.query(Song.id).count()
print('# Songs: ', track_count_total)

session.query(
    Band.name,
    sa.cast(track_counts_by_band.c.song_count, sa.Float) * 100.0 / track_count_total
).join(
    track_counts_by_band,
    track_counts_by_band.c.band_id == Band.id
).all()

# Inserting Records

In [ ]:
import datetime

# Create a new band instance (not tied to DB/session yet)
john_prine = Band(name='John Prine')

# Tell the session to start tracking our new instance.
# No data is sent to the DB until our current 
# transaction is commited or the session is flushed
session.add(john_prine)

# Create an album for our new band
souvenirs = Album()
souvenirs.title = 'Souvenirs'
souvenirs.release_date = datetime.date(2000, 10, 31)
souvenirs.band = john_prine

session.add(souvenirs)

# Create songs for our new album
track_data = [
    ('Souvenirs', 1, 222.866), 
    ('Fish and Whistle', 2, 180.693), 
    ('Far From Me', 3, 309.706),
    ('Angel From Montgomery', 4, 307.626), 
    ('Donald & Lydia', 5, 249.533),
    ('Christmas In Prison', 6, 216.84), 
    ('Storm Windows', 7, 266.733),
    ('Grandpa Was a Carpenter', 8, 172.533), 
    ('The Late John Garfield Blues', 9, 226.293),
    ('Blue Umbrella', 10, 233.24), 
    ("Six O'Clock News", 11, 276.666),
    ("People Puttin' People Down", 12, 240.226), 
    ('Sam Stone', 13, 279.6),
    ("Please Don't Bury Me", 14, 186.8), 
    ('Hello In There', 15, 297.173)
]
for title, track_num, length in track_data:
    track = Song(title=title, track_num=track_num, length_seconds=length)
    souvenirs.songs.append(track)
    # adding the songs to the album's relationship list automatically 
    # adds them to the session

print('About to commit')
session.commit() # Insert statements are executed now

In [ ]:
session.query(Song).all()

## Updating Records

In [ ]:
white_album = session.query(Album).filter(
    Album.title == 'The Beatles (Remastered)'
).first()

# Just update like any model object
white_album.title = 'The White Album'

# No need to call session.add() here. The session is already tracking the 
# object since we got it from a query

# Database record is not updated until the session is commited or flushed
# Executing a select query will automatically flush the session before 
# the query runs
session.query(Album).all()

In [ ]:
# Revert our changes in the current transaction
session.rollback()
print('Session Rolled back')

session.query(Album).all()

## Additional Resources

* [SQLAlchemy Docs and Tutorials](http://docs.sqlalchemy.org/en/latest/orm/tutorial.html)
* [In Depth Introducton by Mike Bayer (Creator of SQLA)](https://www.youtube.com/watch?v=woKYyhLCcnU)